In [134]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
from dateutil import parser as dparser
import urllib.parse
import timeit
import os

import numpy as np
import pandas as pd
import math

from util.corpus import Corpus
from util.config import config
from util.pyRanker import BM25
from util.web_query_v2 import web_query
from util.ticker import Ticker
from util.data_manager import data_manager
import ipywidgets as widgets


#pull api keys from the config file
cfig=config()
#setting data manager
datadir = os.path.join(os.getcwd(),"_data")
manager = data_manager(datadir=datadir)


In [153]:
corpus=Corpus()

In [160]:
#Setting up query
d_start="9/1/2021"
#d_end="11/15/2021"
d_end='Now'
desired_docs='max'
prime_query='both'
pagesizes = {'usearch':50, 'poly':700, 'currents':200 ,'newsapi':100}
pages=2

In [161]:
#initiate ticker
tick = Ticker(config=cfig, t="GM", source='alpha')
print(tick.name)
print(tick.name_adj)

General Motors Company
GeneralMotors


In [162]:

start = timeit.default_timer()


#Define Web_query object
print('API requests')
wq=web_query(config=cfig, pagesizes=pagesizes)

#web_query_v2 for ticker object handling and multi-page api requests (when they work)
wq.query_all(tickerobj=tick, d_start=d_start, d_end=d_end, threaded=True, pages=pages)

#compile results into a singular dataframe
wq.compile_results()
print("Returned Urls:",len(wq.results))

#scrap text from the results urls to form documents
print('Scraping Urls')
wq.scrape_results(threaded=True, max_docs=desired_docs)


#build corpus from web query results
#corpus=Corpus()
#store the web query data frame in the corpus for referencing urls and titles
results = wq.get_results()
corpus.set_results(results)
#assign corpus documents as the web query documents and urls
corpus.set_corpus(documents = wq.documents, urls = wq.urls)

#Initiating Ranker
bm25 = BM25(norm='l2', smooth_idf=True, stopwords=corpus.stopwords, sublinear_tf=True)
bm25.fit(corpus.documents)

#Building Queries
corpus.build_queries(ticker=tick, prime=prime_query)
#Ranking and Pruning
corpus.rank_docs(ranker=bm25)
corpus.prune_docs()

#Sub-Dividing
print('Sub-Dividing Documents')
#corpus.sub_divide(tokenizer=tokenizer, cutoff=2, method='sen')
corpus.sub_divide(cutoff=2, method='sen')
print('Sub-docs:',len(corpus.sub_list))
print('Source count:', len(corpus.sub_docs))

#Ranking and Pruning
corpus.rank_subdocs(ranker=bm25)
corpus.prune_subdocs()

print("Number of sub_docs:",len(corpus.sub_list))
t = np.array(corpus.sub_list_scores)
z=np.where(t==0.0)
print("Number zero ranked:", len(z[0]))
print("percent of useless subdocs:", round((len(z[0])/len(corpus.sub_list))*100,2),"%")

#Relevant Set
corpus.make_relevant()
corpus.rank_relevant(ranker=bm25)
#shouldn't be any zeros because of the pruned subdocs, but cant hurt
corpus.prune_relevant(method='finite',cutoff=0.0)
print('relevant sources:',len(corpus.relevant_set))
print('relevant sub_docs:',len(corpus.rel_list))

#Sentiments
print('Getting Sentiments')
corpus.get_sentiments()

#Building Dataset
print('Storing Data')
corpus.data_preprocess()
corpus.build_fulldf()
corpus.build_pricedf(ticker=tick)

stop = timeit.default_timer()
print('RunTime:',round((stop-start)/60,2),'mins')

API requests
NewsAPI: 2 page: 1
Polygon: 700
Usearch: 50 page: 1
error on Newsapi page: 2
Usearch: 50 page: 2
Returned Urls: 766
Scraping Urls


IntProgress(value=0, max=766)

Sub-Dividing Documents


IntProgress(value=0, max=715)

Token indices sequence length is longer than the specified maximum sequence length for this model (661 > 512). Running this sequence through the model will result in indexing errors


Sub-docs: 6385
Source count: 709
Number of sub_docs: 6385
Number zero ranked: 5204
percent of useless subdocs: 81.5 %
relevant sources: 659
relevant sub_docs: 1181
Getting Sentiments


IntProgress(value=0, max=1181)

Storing Data


C:\Users\Tony\anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



RunTime: 7.44 mins


In [163]:
manager.store_data(ticker=tick.ticker, full_df=corpus.full_df, price_df=corpus.price_df)
manager.get_ticker_list()
manager.get_data_info(ticker=tick.ticker)

{'full_df': {'mindate': Timestamp('2021-07-15 00:00:00'),
  'maxdate': Timestamp('2021-12-05 00:00:00'),
  'rows': 659},
 'price_df': {'mindate': Timestamp('2021-07-15 00:00:00'),
  'maxdate': Timestamp('2021-12-03 00:00:00'),
  'rows': 98}}

In [164]:
price_df = manager.get_pricedf(ticker=tick.ticker)
price_df = price_df.sort_values(by='pub_date')

In [165]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=3, cols=1, shared_xaxes=True,vertical_spacing=0.02, subplot_titles=(tick.ticker,'Sentiment_Score', 'Used News Volume'), 
               row_width=[0.4, 0.6, 1.5], specs=[[{"secondary_y": True}],[{"secondary_y": True}],[{"secondary_y": True}]])
fig.add_trace(go.Candlestick(x=price_df['pub_date'],
                open=price_df['open'], high=price_df['high'],
                low=price_df['low'], close=price_df['close'], name="Price"),
               secondary_y=True, row=1,col=1)

fig.add_trace(go.Bar(x=price_df['pub_date'], y=price_df['volume'],opacity=0.2, name='Volume'),secondary_y=False)
fig.add_trace(go.Line(x=price_df['pub_date'], y=price_df['scores'], showlegend=False), row=2, col=1)
#fig.add_trace(go.Line(x=price_df['pub_date'], y=price_df['sentiments'], showlegend=False), row=2, col=1)
#fig.add_trace(go.Line(x=price_df['pub_date'], y=price_df['relevance'], showlegend=False), row=2, col=1)
fig.add_trace(go.Bar(x=price_df['pub_date'],y=price_df['Doc_Volume'],opacity=0.2, name='Used News Volume'),row=3, col=1)
fig.update_layout(xaxis_rangeslider_visible=False, width=1000, height=750)
fig.layout.yaxis2.showgrid=False
fig.show()

C:\Users\Tony\anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


